In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets torch torchvision

In [ ]:
!pip install transformers bitsandbytes datasets torch torchvision

In [1]:
!pip install -U accelerate
!pip install -U transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install rouge_score
!pip install torch
!pip install torchvision
!pip install bitsandbytes
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 399.8 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 2.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0004623f5ea3a2aa72d61c06b656ce526a4bed07a582f67a5c05c67f334399ad
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.9 MB/s eta 0:00:0000:0100:01
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'



In [ ]:
!pip install evaluate

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from datasets import load_dataset

dataset = load_dataset("Seeker38/augmented_vi_face_wiki", split="train")

Generating train split:   0%|          | 0/9525 [00:00<?, ? examples/s]

# Fine Tune ViT-PhoBERT model


In [4]:
words_to_remove = ["Bản tin:", "Bản tin", "bản tin", "Tập tin", "@@", "Tập tin:", "Juliana", "jpg", "png", "tập tin"]

# Function to remove specified words/phrases from text
def clean_text(text, words_to_remove):
    for word in words_to_remove:
        text = text.replace(word, "")
    return text

dataset = dataset.map(lambda x: {"text": clean_text(x["text"], words_to_remove)})

Map:   0%|          | 0/9525 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel
from PIL import Image
import requests

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# decoder_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", add_special_tokens=True)

phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", add_special_tokens=True)

2024-06-06 09:50:03.238008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 09:50:03.238138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 09:50:03.361580: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
if phobert_tokenizer.pad_token is None:
    phobert_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model detail

In [ ]:
batch_size = 32
epochs = 20

In [ ]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("google/vit-base-patch16-224-in21k", "vinai/phobert-base-v2")

# ids
# Set tokenizer parameters
model.config.decoder_start_token_id = phobert_tokenizer.cls_token_id
model.config.pad_token_id = phobert_tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [13]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from PIL import Image
from datasets import load_dataset
from torch.utils.data import Dataset


In [23]:
class ImageCaptionDataset(Dataset):
    def __init__(self, dataset, tokenizer, image_height, image_width):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.image_height = image_height
        self.image_width = image_width

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        image = self.dataset[index]['image']
        image = image.resize((self.image_width, self.image_height))
        image = np.array(image, dtype=np.uint8)
        image = torch.tensor(np.moveaxis(image, -1, 0), dtype=torch.float32)

        caption = self.dataset[index]['text']
        encoded_caption = self.tokenizer(caption, add_special_tokens=True, padding="max_length", max_length=70, truncation=True)
        caption_token = encoded_caption.input_ids
        attention_mask = encoded_caption.attention_mask

        # Replace pad token id with -100 to ignore these tokens in the loss calculation
        caption_token = [token if token != self.tokenizer.pad_token_id else -100 for token in caption_token]

        return {
            "pixel_values": image,
            "caption_token": torch.tensor(caption_token, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long)
        }

In [ ]:
type(dataset[0]["image"])

In [24]:
# Initialize datasets
train_dataset = ImageCaptionDataset(dataset, phobert_tokenizer, image_height=224, image_width=224)
val_dataset = ImageCaptionDataset(dataset, phobert_tokenizer, image_height=224, image_width=224)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(40, 10))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    
    # Get the image and move the axes to match the expected shape for imshow
    image = train_dataset.__getitem__(i)['pixel_values'].numpy()
    image = np.moveaxis(image, 0, -1)  # Convert from (C, H, W) to (H, W, C)
    
    # Ensure the image data is in the correct range (0-255)
    image = image.astype(np.uint8)
    
    # Display the image
    plt.imshow(image)
    plt.axis('off')
    
    # Decode the caption and handle special tokens
    caption_token_ids = train_dataset.__getitem__(i)['caption_token'].numpy()
    decoded_caption = phobert_tokenizer.decode([token if token != -100 else 0 for token in caption_token_ids])
    caption = " ".join([word for word in decoded_caption.split() if word not in phobert_tokenizer.all_special_tokens])
    
    # Add the caption text
    plt.text(0, 230, caption, fontsize=20, color='white', backgroundcolor='black', wrap=True)
    
plt.gcf().set_size_inches(40, 30)
plt.show()

In [25]:
from torch.utils.data import DataLoader
# Define data loaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
from transformers import  AdamW, get_scheduler
epochs = 20

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(train_dataloader)
# num_training_steps = len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=int(num_training_steps/5), num_training_steps=num_training_steps)


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [ ]:
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
    print("yes")
else:
    print("no")

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(epochs * len(train_dataloader)))

model.train()
for epoch in range(epochs):
    losses = []
    for batch in train_dataloader:
        # Move batch to the correct device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model(
            pixel_values=batch['pixel_values'],
            labels=batch['caption_token'],
            decoder_attention_mask=batch['attention_mask']
        )
        
        
        loss = outputs.loss.mean()  # Ensure the loss is a scalar
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        losses.append(loss.item())

    avg_loss = sum(losses) / len(losses)
    print(f'Epoch: {epoch + 1}, Loss: {avg_loss:.4f}')

In [ ]:
# Saving the model, including its configuration
model.save_pretrained("Image_Captioning_Fine_Tune_ViT_PHOBERT_model") 

# EVALUATE

In [15]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk

In [ ]:
def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    bleu_scores = []
    rouge_scores = []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model.generate(pixel_values=batch['pixel_values'])
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(batch['caption_token'], skip_special_tokens=True)

            for pred, label in zip(decoded_preds, decoded_labels):
                reference = [label.split()]
                candidate = pred.split()
                bleu_scores.append(sentence_bleu(reference, candidate))
                rouge_score = scorer.score(' '.join(reference[0]), ' '.join(candidate))
                rouge_scores.append(rouge_score)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

    print(f'Average BLEU Score: {avg_bleu:.4f}')
    print(f'Average ROUGE-1 Score: {avg_rouge1:.4f}')
    print(f'Average ROUGE-2 Score: {avg_rouge2:.4f}')
    print(f'Average ROUGE-L Score: {avg_rougeL:.4f}')

    model.train()

In [ ]:
# Generate caption function
def generate_caption(image, model, tokenizer, device):
    model.eval()
    image = image.resize((224, 224))
    image = np.array(image, dtype=np.uint8)
    image = torch.tensor(np.moveaxis(image, -1, 0)).unsqueeze(0).to(device)

    with torch.no_grad():
        output_ids = model.generate(pixel_values=image)
        caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    model.train()
    return caption

In [ ]:
# Evaluate the model
evaluate_model(model, val_dataloader, phobert_tokenizer)

In [ ]:
def generate_caption(image, model):

    image = torch.unsqueeze( torch.tensor(image), 0)
    predictions = phobert_tokenizer.decode(model.generate(pixel_values=image.to(device))[0],skip_special_tokens=True)
    caption = predictions.replace(".","")


    image = np.moveaxis(image[0].numpy(),0,-1)
    plt.figure(figsize=(4,4))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print( " Generated Caption:  ", caption )
    return caption

pred_caption = generate_caption(image=train_dataset.__getitem__(0)['pixel_values'].numpy(), model=model)